In [129]:
import skrf as rf
from math import pi
from skrf.media.cpw import CPW
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from modeling.utils.resonators import cpw_cl_ll, resonator_kappa, cpw_kinetic_inductance, penetration_depth
from modeling.utils.network import assemble_network
from modeling.utils.qubits import *
from qucat import Network,L,J,C,R
from modeling.clockmon.libraries.clockmon_library import clockmon_coupling_libraries, clockmon_library, clockmon_cqr_to_ground
from modeling.smooth_capacitor.libraries.smooth_capacitor_library import digit_tee_library, digit_tee_ck_to_ground, get_ck
import scqubits as scq
from scqubits import ParameterSweep

In [130]:
rr_freq_target = 7e9
a = 10e-6
b = 6e-6
ep_r = 11.45
Z0 = CPW(w = a, s = b, ep_r = ep_r).z0[0].real
l_r = Z0*pi/(4*pi*rr_freq_target)
c_r = 1/((2*pi*rr_freq_target)**2*l_r)

In [131]:
Lj = 10e-9

C11 = 20e-15
C12 = 10e-15
C13 = 2e-15
C14 = 1e-15

C22 = 50e-15
C23 = 110e-15
C24 = 3e-15

C33 = 50e-15
C34 = 9e-15

C44 = 19e-15

In [132]:
c_sig_1port = ((C33 + C13)*(C22 + C12))/(C33 + C22 + C13 + C12) + C23
c_qr_1port = (C12*C33 - C13*C22) / (C22 + C12 + C33 + C13)
print(c_sig_1port)
print(c_qr_1port)
print("1port_frequency = ", 1/np.sqrt(Lj*c_sig_1port)/2/np.pi)

1.3785714285714284e-13
3.571428571428572e-15
1port_frequency =  4286526322.6082716


In [133]:
cir = Network([
    C(0, 1, C11),
    C(1, 2, C12),
    C(1, 3, C13),
    C(1, 4, C14),
    C(2, 3, C23),
    C(2, 0, C22),
    C(2, 4, C24),
    C(0, 3, C33),
    C(3, 4, C34),
    J(2, 3, 'Lj'),
    C(0, 4, C44),
    ])
f,k,A,chi = cir.f_k_A_chi(pretty_print=True, Lj = Lj)
ham = cir.hamiltonian(modes = [0],
                    taylor = 12,
                    excitations = [10],
                    Lj = Lj)
ee = ham.eigenenergies()
ee -= ee[0]
print(ee)

c_sigma_extracted = 1/Lj/(2*np.pi*f[0])**2
print("C_sigma extracted = ", c_sigma_extracted)

         mode |       freq.  |       diss.  |       anha.  |
            0 |     4.25 GHz |          0Hz |      138 MHz |

Kerr coefficients (diagonal = Kerr, off-diagonal = cross-Kerr)
         mode |         0    |
            0 |      138 MHz |

[0.00000000e+00+0.j 4.10673614e+09+0.j 8.06346683e+09+0.j
 1.18607917e+10+0.j 1.54923046e+10+0.j 1.89845812e+10+0.j
 2.24744187e+10+0.j 2.63120609e+10+0.j 3.10724658e+10+0.j
 3.70873335e+10+0.j]
C_sigma extracted =  1.402521457102301e-13


In [134]:
cir = Network([
    C(0,1,c_sigma_extracted- c_qr_1port), # transmon
    J(0,1,'Lj'),
    C(0,2,c_r - c_qr_1port), # resonator
    L(0,2,l_r),
    C(1,2, c_qr_1port*0.01), # coupling capacitor
    R(0, 2, 1e6)
    ])
# f,k,A,chi = 
cir.f_k_A_chi(pretty_print=True, Lj = Lj)
ham = cir.hamiltonian(modes = [0, 1],
                    taylor = 6,
                    excitations = [10, 10],
                    Lj = Lj)
ee = ham.eigenenergies()
ee -= ee[0]
f

         mode |       freq.  |       diss.  |       anha.  |
            0 |      4.3 GHz |          0Hz |      142 MHz |
            1 |     7.04 GHz |      567 kHz |      379 nHz |

Kerr coefficients (diagonal = Kerr, off-diagonal = cross-Kerr)
         mode |         0    |         1    |
            0 |      142 MHz |              |
            1 |      14.7 Hz |      379 nHz |



array([4.24976949e+09])

In [135]:
cir.f_k_A_chi(Lj = Lj)


(array([4.30437173e+09, 7.04392937e+09]),
 array([     0.        , 566746.35123737]),
 array([1.41681783e+08, 3.78827978e-07]),
 array([[1.41681783e+08, 1.46523750e+01],
        [1.46523750e+01, 3.78827978e-07]]))

# One port

In [136]:
c_sig = c_sig_1port
cir = Network([
    C(0,1,c_sig), # transmon
    J(0,1,'Lj'),
    ])
# f,k,A,chi = 
cir.f_k_A_chi(pretty_print=True, Lj = Lj)
ham = cir.hamiltonian(modes = [0],
                    taylor = 6,
                    excitations = [10],
                    Lj = Lj)
ee = ham.eigenenergies()
ee -= ee[0]
freq_harm = 1/np.sqrt(Lj*c_sig)/2/np.pi
ee

         mode |       freq.  |       diss.  |       anha.  |
            0 |     4.29 GHz |          0Hz |      141 MHz |

Kerr coefficients (diagonal = Kerr, off-diagonal = cross-Kerr)
         mode |         0    |
            0 |      141 MHz |



array([0.00000000e+00+0.j, 4.14109767e+09+0.j, 8.12994932e+09+0.j,
       1.19577356e+10+0.j, 1.56192098e+10+0.j, 1.91420962e+10+0.j,
       2.26630668e+10+0.j, 2.65330197e+10+0.j, 3.13339168e+10+0.j,
       3.74048252e+10+0.j])

In [137]:
1/np.sqrt(Lj*c_sig)/2/np.pi

4286526322.6082716

In [138]:
CA = c_sig_1port
CB = c_r
CC = c_qr_1port
det = CA*CB + CA*CC + CB*CC
CC_eff = det/2/CC

In [139]:
CC

3.571428571428572e-15

In [140]:
CA + CB*CC/(CB + CC)

1.413842720086158e-13

In [141]:
c_sigma_extracted

1.402521457102301e-13

In [142]:
c_sigma_extracted

1.402521457102301e-13

In [143]:
# Just floating qubit
C11 = 70e-15
C22 = 69e-15
C12 = 100e-15

In [144]:
c_sigma_extracted = 90e-15
cqrs = 3.98953836e-15
c_r = 2.84357895e-13
l_r = 1.81793726e-09
Lj = 1.57443917e-08
cir = Network([
    C(0,1,c_sigma_extracted- cqrs), # transmon
    J(0,1,'Lj'),
    C(0,2,c_r - cqrs), # resonator
    L(0,2,l_r),
    C(1,2, cqrs), # coupling capacitor
    R(0, 2, 1e6)
    ])
# f,k,A,chi = 
cir.f_k_A_chi(pretty_print=True, Lj = Lj)
ham = cir.hamiltonian(modes = [0, 1],
                    taylor = 6,
                    excitations = [10, 10],
                    Lj = Lj)

# Diagonalize the Hamiltonian
eigenvalues, eigenstates = ham.eigenstates()

# Sort eigenvalues
eigenvalues = np.sort(eigenvalues)

# Calculate dispersive shift
dispersive_shift = eigenvalues[1] - eigenvalues[0]

# Calculate Kerr shift
kerr_shift = eigenvalues[2] - 2 * eigenvalues[1] + eigenvalues[0]

print(f"Dispersive Shift: {dispersive_shift}")
print(f"Kerr Shift: {kerr_shift}")


         mode |       freq.  |       diss.  |       anha.  |
            0 |     4.23 GHz |       115 Hz |      215 MHz |
            1 |        7 GHz |      560 kHz |       186 Hz |

Kerr coefficients (diagonal = Kerr, off-diagonal = cross-Kerr)
         mode |         0    |         1    |
            0 |      215 MHz |              |
            1 |      400 kHz |       186 Hz |

Dispersive Shift: (4000000004.7309394-2.1249843765904166e-07j)
Kerr Shift: (-996768842.7122467-1.749608007044502e-07j)


In [145]:
import numpy as np
from qutip import *

# Parameters
omega_r = 7e9 * 2 * np.pi  # Resonator frequency in Hz
omega_q = 4e9 * 2 * np.pi  # Qubit frequency in Hz
E_c = 215224769.19932842 *2*np.pi    # Charging energy in Hz
g = 4.26221746e+08      # Coupling strength in Hz

# Number of levels
N_r = 10   # Number of resonator levels
N_q = 5    # Number of qubit levels

# Annihilation operators
a = tensor(destroy(N_r), qeye(N_q))  # Resonator
b = tensor(qeye(N_r), destroy(N_q))  # Qubit

# Hamiltonian components
H_r = omega_r * a.dag() * a
H_q = omega_q * b.dag() * b - (E_c / 2) * b.dag() * b * (b.dag() * b - 1)
H_coupling = g * (a.dag() * b + a * b.dag())

# Total Hamiltonian
H = H_r + H_q + H_coupling

# Diagonalize the Hamiltonian
eigenvalues, eigenstates = H.eigenstates()

# Sort eigenvalues
eigenvalues = np.sort(eigenvalues)

# Calculate dispersive shift
dispersive_shift = eigenvalues[1] - eigenvalues[0]

# Calculate Kerr shift
kerr_shift = eigenvalues[2] - 2 * eigenvalues[1] + eigenvalues[0]

print(f"Dispersive Shift: {dispersive_shift}")
print(f"Kerr Shift: {kerr_shift}")


Dispersive Shift: 25123108524.98051
Kerr Shift: -6254287195.966087


In [146]:
dispersive_shift/2/np.pi

3998466907.584784

In [147]:
kerr_shift/2/np.pi

-995400722.7543523

In [148]:
eigenvalues[2]/2/np.pi - eigenvalues[1]/2/np.pi

3003066184.8304315

In [149]:
cir.eigenfrequencies(Lj =Lj)

array([4.22725480e+09, 7.00342754e+09])

In [166]:
import scqubits as scq
import numpy as np

    # C(0,1,c_sigma_extracted- cqrs), # transmon
    # J(0,1,'Lj'),
    # C(0,2,c_r - cqrs), # resonator
    # L(0,2,l_r),
    # C(1,2, cqrs), # coupling capacitor
    
zp_yaml = """# zero-pi
branches:
- ["JJ", 0,1, EJ = 1.57443917e-08, 20]
- ["L", 0,2, 1.81793726e-09]
- ["C", 0,1, 8.601046163999999e-14]
- ["C", 0,2, 2.8036835664e-13]
- ["C", 1,2, 3.98953836e-15]
"""
zero_pi = scq.Circuit(zp_yaml, from_file=False)
zero_pi.hamiltonian()



<108900x108900 sparse matrix of type '<class 'numpy.complex128'>'
	with 5121684 stored elements in Compressed Sparse Column format>

In [170]:
cqrs

3.98953836e-15

In [161]:
zero_pi.hamiltonian()


<9900x9900 sparse matrix of type '<class 'numpy.complex128'>'
	with 138780 stored elements in Compressed Sparse Column format>

In [163]:
zero_pi.eigenvals()


array([-3.66713832, -3.65542458, -3.64979332, -3.63807958, -3.54945453,
       -3.54891778])